In [141]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout,BatchNormalization
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import os
from pyvi import ViTokenizer
import string
from tqdm import tqdm

In [142]:
!pip install pyvi

In [143]:
filename = "/content/hoaVangTrenCoXanh.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [144]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    
    tokens = doc.split() #Split in_to words
    
    table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
    tokens = [w.translate(table) for w in tokens]
    
    tokens = [word for word in tokens if word]
    
    return tokens

In [145]:
words = clean_document(raw_text)

In [146]:
words[:10]

['chú', 'đàn', 'bảo', 'tôi', 'con', 'xòe', 'tay', 'ra', 'cho', 'chú']

In [147]:
# đưa vào set lấy các trường hợp trùng 
words_set = sorted(list(set(words)))
#lấy index
char_to_int = dict((c, i) for i, c in enumerate(words_set))
int_to_char = dict((i, c) for i, c in enumerate(words_set))

In [148]:
n_chars = len(words)
n_vocab = len(words_set)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  13654
Total Vocab:  2095


In [149]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = words[i:i + seq_length]
	seq_out = words[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
n_patterns

13554

In [150]:
X = np.reshape(dataX, (len(dataX), seq_length, 1))
#co dan X
X = X / float((n_vocab))
#one hot encoding
y = np_utils.to_categorical(dataY)

In [157]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [158]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_7 (Dense)              (None, 2095)              1074735   
Total params: 4,228,655
Trainable params: 4,227,631
Non-trainable params: 1,024
________________________________________

In [159]:
filepath="model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
106/106 [==============================] - 14s 86ms/step - loss: 7.5626 - accuracy: 0.0071

Epoch 00001: loss improved from inf to 7.40094, saving model to model.hdf5
Epoch 2/200
106/106 [==============================] - 9s 87ms/step - loss: 6.6022 - accuracy: 0.0423

Epoch 00002: loss improved from 7.40094 to 6.52845, saving model to model.hdf5
Epoch 3/200
106/106 [==============================] - 9s 89ms/step - loss: 6.3265 - accuracy: 0.0442

Epoch 00003: loss improved from 6.52845 to 6.34200, saving model to model.hdf5
Epoch 4/200
106/106 [==============================] - 9s 90ms/step - loss: 6.1654 - accuracy: 0.0409

Epoch 00004: loss improved from 6.34200 to 6.22266, saving model to model.hdf5
Epoch 5/200
106/106 [==============================] - 10s 91ms/step - loss: 6.0589 - accuracy: 0.0424

Epoch 00005: loss improved from 6.22266 to 6.12235, saving model to model.hdf5
Epoch 6/200
106/106 [==============================] - 10s 93ms/step - loss: 5.9494 - accura

In [160]:
filename = 'model.hdf5'
model.load_weights(filename)
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ' '.join([int_to_char[value] for value in pattern]), "\"")
for i in range(50):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    print(result)
    pattern.append(index)
    pattern = pattern[1:]
print ("\nDone.")

Seed:
" nói rất hay điều đơn_giản thế_mà lâu_nay chú không nghĩ ra ha_ha hay quá thế là tôi vẫn có năm cái hoa_tay bà_con ơi tôi xác_nhận với các bạn là chú đàn vẫn còn năm cái hoa_tay dù không ai được nhìn thấy chúngnữa vì chú vẽ rất đẹp dưới mắt thằng tường thì tôi là người vẽ đẹp nhất nhưng dưới mắt tôi thì chú đàn mới là người số một các bài_tập vẽ của tôi lúc mê chơi tôi vẫn thường nhờ chú vẽ giùm và bao giờ các tranh vẽ của chú cũng đạt điểm_cao nhất lớp "
hồi
đó
nhà
tôi
ở
cạnh
đường
quốc_lộ
sau
nhà
là
một
nghĩa_trang
rộng
mênh_mông
nghĩa_trang
rộng
đến
mức
ngay
trong
vườn
nhà
tôi
cũng
có
hai
ngôi
mộ
hoang
trú_ngụ
từ
rất
lâu
trước
khi
gia_đình
tôi
dọn
đến
buổi
tối
ngồi
trong
nhà
nhìn
ra
những
đốm
nhang

Done.
